In [1]:
from __future__ import print_function
import tensorflow as tf
import keras # this is needed
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
# from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import os

Using TensorFlow backend.


In [3]:
number_classes = 5
image_rows,image_columns= 48, 48
batch_size = 32

In [4]:
# The absolute path of your training/validation data
train_data_path = 'fer2013/train'
validation_data_path = 'fer2013/validation'

In [5]:
# train data generator
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=30,
                    shear_range=0.3,
                    zoom_range=0.3,
                    width_shift_range=0.4,
                    height_shift_range=0.4,
                    horizontal_flip=True,
                    fill_mode='nearest')

In [6]:
# validation data generator
validation_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
# Train generator
train_generator = train_datagen.flow_from_directory(
                    train_data_path,
                    color_mode='grayscale',
                    target_size=(image_rows,image_columns),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True)

Found 24255 images belonging to 5 classes.


In [8]:
#validation generator
validation_generator = validation_datagen.flow_from_directory(
                          validation_data_path,
                          color_mode='grayscale',
                          target_size=(image_rows,image_columns),
                          batch_size=batch_size,
                          class_mode='categorical',
                          shuffle=True)

Found 3006 images belonging to 5 classes.


In [9]:
# define our model you can name it whatever you want
model = Sequential()

In [10]:
# layer 1
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(image_rows,image_columns,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(image_rows,image_columns,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
# layer 2
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [12]:
# layer 3
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [13]:
# layer 4
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [14]:
# layer 5
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [15]:
# layer 6
model.add(Dense(number_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [16]:
# Display the total neurons in our model named model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0         
__________

In [17]:
# checkpoint 
checkpoint = ModelCheckpoint('Emotion_little_vgg.h5',
                              monitor='val_loss',
                              mode='min',
                              save_best_only=True,
                              verbose=1)

In [18]:
# early stopping
earlystop = EarlyStopping(monitor='val_loss', 
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True)

In [19]:
# reduce linear regression on plateau
reduce_lr= ReduceLROnPlateau(monitor='va;_loss',
                             factor=0.2,
                             patience=3,
                             verbose=1,
                             min_delta=0.0001)

In [20]:
# callbacks
callbacks = [earlystop,checkpoint,reduce_lr]

In [21]:
# compile our model and optimize using Adam 
model.compile(loss='categorical_crossentropy', optimizer=adam(lr=0.001), metrics=['accuracy'])

In [22]:
# number of samples respectively 
nb_train_samples=24176
nb_validation_samples=3006
# Number of iterations 
# NOTE be careful with adjusting the epoch 
# this takes real long to complete
epochs=3

In [27]:
# for back propagation
history=model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples//batch_size,
            epochs=epochs,
            callbacks=callbacks,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples//batch_size)

Epoch 1/3
755/755 [==============================] - 693s 917ms/step - loss: 1.7564 - acc: 0.2512 - val_loss: 1.5684 - val_acc: 0.2872

Epoch 00001: val_loss improved from inf to 1.56839, saving model to Emotion_little_vgg.h5


C:\Program Files\Anaconda3\lib\site-packages\keras\callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `va;_loss` which is not available. Available metrics are: val_loss,val_acc,loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 2/3
755/755 [==============================] - 430s 569ms/step - loss: 1.5663 - acc: 0.2907 - val_loss: 1.5405 - val_acc: 0.3043

Epoch 00002: val_loss improved from 1.56839 to 1.54055, saving model to Emotion_little_vgg.h5
Epoch 3/3
755/755 [==============================] - 422s 559ms/step - loss: 1.5464 - acc: 0.3004 - val_loss: 1.5218 - val_acc: 0.3157

Epoch 00003: val_loss improved from 1.54055 to 1.52184, saving model to Emotion_little_vgg.h5
